In [1]:
import urllib.request
from selenium import webdriver
import bs4
import requests

url="https://page.kakao.com/home?seriesId=54811340"
driver = webdriver.Chrome("C:/chromedriver.exe")
driver.get(url)
bs_obj = bs4.BeautifulSoup(driver.page_source, "html.parser")
ul = bs_obj.find("div", {"class": "jsx-3063119961 textBox"})
print(ul)

None


In [1]:
import urllib.request
from selenium import webdriver
import bs4
import requests
from urllib import parse

print("작품이름 -> ")
search_name=input()
tmp = search_name
search_name_nonspace=search_name.replace(" ","")

url = "http://page.kakao.com/search?word=" + parse.quote(tmp)
driver = webdriver.Chrome("C:/chromedriver.exe")
driver.get(url)

bs_obj = bs4.BeautifulSoup(driver.page_source, "html.parser")
boxes = bs_obj.findAll("div", {"class": "css-151xn98"})

for box in boxes 
    title = box.find("div", {"class": "text-ellipsis css-11602e0"}).text
    if (search_name or search_name_nonspace) not in title:
        continue
    elif ("단행본" in title):
        continue
    elif
    
    
#기다무소설or 소설
#[단행본] X
#[닥터최태수 or 닥터 최태수 포함]

작품이름 -> 
닥터 최태수
닥터 최태수 [매주 수요일 연재]
약사 클라라 [독점연재]
사람답게 살자 [독점연재]
메디컬 싸인 [독점연재]
23시 59분[독점연재]
사랑은 오늘도 맛있다 [단독선공개]
난 할 수 있어
닥터 최태수
소현세자의 첫사랑 [연재]
닥터 최태수 [단행본]


In [6]:
import urllib.request
from selenium import webdriver
import bs4
import requests

url="https://page.kakao.com/home?seriesId=47495746"

driver = webdriver.Chrome("C:/chromedriver.exe")
driver.get(url)
bs_obj = bs4.BeautifulSoup(driver.page_source, "html.parser")

driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[2]/button[1]').click()

title=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[1]/h2')
print("제목 : " + title.text)

update_days = driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[1]/p[1]')
print("연재일 : " + update_days.text)

writer=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[1]/p[2]')
print("작가 : " + writer.text)



genre=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[2]/div[2]')
print("장르 : " + genre.text)

age=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[4]/div[2]')
print("연령등급 : " + age.text)

publisher=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[3]/div[2]')
print("출판사 : " + publisher.text)



제목 : 닥터 최태수
연재일 : 연재 중
작가 : 조석호
장르 : 기다무소설현판
연령등급 : 전체이용가
출판사 : 시나브로


'\nmonopoly=driver.find_element_by_xpath(\'\')\nprint("독점여부 : " + monopoly)\nenjoyer=driver.find_element_by_xpath(\'\')\nprint("감상인원 : " + enjoyer)\n'

In [9]:
import urllib.request
from selenium import webdriver
import bs4
import requests
from urllib import parse

print("작품이름 -> ")
search_name=input()
tmp = search_name
search_name_nonspace=search_name.replace(" ","")

url = "http://page.kakao.com/search?word=" + parse.quote(tmp)
driver = webdriver.Chrome("C:/chromedriver.exe")
driver.get(url)

bs_obj = bs4.BeautifulSoup(driver.page_source, "html.parser")
boxes = bs_obj.findAll("div", {"class": "css-c09e5i"})

for box in boxes:
    is_novel=box.find("div",{"class":"css-vurnku"}).text
    if("소설" in is_novel):
        parent=box.parent.parent
        title=parent.find("div", {"class": "text-ellipsis css-11602e0"}).text.replace(" ","")
        if("단행본" in title):
            continue
        elif(search_name_nonspace not in title):
            continue
        else:
            main=parent.parent
            monopoly=True
    else:
        continue
    
title=main.find("div", {"class": "text-ellipsis css-11602e0"}).text

if '[' in title:
    print("작품이름 : " + title.split('[')[0])
    print("독점여부 : " + title.split('[')[1][:-1])
else:
    print("작품이름 : " + title)
    print("독점여부 : 미독점")

print("감상인원 : " + main.find('div',{'class','css-zlhhis'}).text)
    
#기다무소설or 소설
#[단행본] X
#[닥터최태수 or 닥터 최태수 포함]

작품이름 -> 
약사클라라
작품이름 : 약사 클라라 
독점여부 : 독점연재
감상인원 : 23만명


In [23]:
import urllib.request
from selenium import webdriver
import bs4
import requests
from urllib import parse
import time


print("작품이름 -> ")
search_name=input()
tmp = search_name
search_name_nonspace=search_name.replace(" ","")

url = "https://page.kakao.com/search?word=" + parse.quote(tmp)
driver = webdriver.Chrome("C:/chromedriver.exe")
driver.get(url)

bs_obj = bs4.BeautifulSoup(driver.page_source, "html.parser")
boxes = bs_obj.findAll("div", {"class": "css-c09e5i"})

for box in boxes:
    is_novel=box.find("div",{"class":"css-vurnku"}).text
    if("소설" in is_novel):
        parent=box.parent.parent
        title=parent.find("div", {"class": "text-ellipsis css-11602e0"}).text.replace(" ","")
        if("단행본" in title):
            continue
        elif(search_name_nonspace not in title):
            continue
        else:
            main=parent.parent.parent
            monopoly=True
    else:
        continue

title=main.find("div", {"class": "text-ellipsis css-11602e0"}).text

if '[' in title:
    print("작품이름 : " + title.split('[')[0])
    print("독점여부 : " + title.split('[')[1][:-1])
else:
    print("작품이름 : " + title)
    print("독점여부 : 미독점")

print("감상인원 : " + main.find('div',{'class','css-zlhhis'}).text)


pages=set()

for link in main.findAll("a", href = re.compile('^(/home)((?!:).)*$')):
    if 'href' in link.attrs: # 위에서 찾은 link에 href 속성이 있는지 확인   
        if link.attrs['href'] not in pages: # 새로운 페이지인지 확인                 
            newPage = link.attrs['href']
    
#기다무소설or 소설
#[단행본] X
#[닥터최태수 or 닥터 최태수 포함]

#작품의 링크
url="https://page.kakao.com" + newPage 
driver = webdriver.Chrome("C:/chromedriver.exe")
driver.get(url)

driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[2]/button[1]').click()

title=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[1]/h2')
print("제목 : " + title.text)

update_days = driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[1]/p[1]')
print("연재일 : " + update_days.text)

writer=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[1]/p[2]')
print("작가 : " + writer.text)



genre=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[2]/div[2]')
print("장르 : " + genre.text)

age=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[4]/div[2]')
print("연령등급 : " + age.text)

publisher=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[3]/div[2]')
print("출판사 : " + publisher.text)

driver.quit()

작품이름 -> 
닥터 최태수
작품이름 : 닥터 최태수
독점여부 : 미독점
감상인원 : 190.4만명
제목 : 닥터 최태수
연재일 : 연재 중
작가 : 조석호
장르 : 기다무소설현판
연령등급 : 전체이용가
출판사 : 시나브로


In [25]:
import urllib.request
from selenium import webdriver
import bs4
import requests
from urllib import parse
import time


print("작품이름 -> ")
search_name=input()
tmp = search_name
search_name_nonspace=search_name.replace(" ","")

url = "https://page.kakao.com/search?word=" + parse.quote(tmp)
driver1 = webdriver.Chrome("C:/chromedriver.exe")
driver1.get(url)

bs_obj = bs4.BeautifulSoup(driver1.page_source, "html.parser")
boxes = bs_obj.findAll("div", {"class": "css-c09e5i"})

for box in boxes:
    is_novel=box.find("div",{"class":"css-vurnku"}).text
    if("소설" in is_novel):
        parent=box.parent.parent
        title=parent.find("div", {"class": "text-ellipsis css-11602e0"}).text.replace(" ","")
        if("단행본" in title):
            continue
        elif(search_name_nonspace not in title):
            continue
        else:
            main=parent.parent.parent
            monopoly=True
    else:
        continue

title=main.find("div", {"class": "text-ellipsis css-11602e0"}).text

if '[' in title:
    print("작품이름 : " + title.split('[')[0])
    print("독점여부 : " + title.split('[')[1][:-1])
else:
    print("작품이름 : " + title)
    print("독점여부 : 미독점")

print("감상인원 : " + main.find('div',{'class','css-zlhhis'}).text)


pages=set()

for link in main.findAll("a", href = re.compile('^(/home)((?!:).)*$')):
    if 'href' in link.attrs: # 위에서 찾은 link에 href 속성이 있는지 확인   
        if link.attrs['href'] not in pages: # 새로운 페이지인지 확인                 
            newPage = link.attrs['href']
    
time.sleep(1)
driver1.quit()
#기다무소설or 소설
#[단행본] X
#[닥터최태수 or 닥터 최태수 포함]

#작품의 링크
url="https://page.kakao.com" + newPage 
driver2 = webdriver.Chrome("C:/chromedriver.exe")
driver2.get(url)

driver2.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[2]/button[1]').click()

title=driver2.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[1]/h2')
print("제목 : " + title.text)

update_days = driver2.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[1]/p[1]')
print("연재일 : " + update_days.text)

writer=driver2.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[1]/p[2]')
print("작가 : " + writer.text)



genre=driver2.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[2]/div[2]')
print("장르 : " + genre.text)

age=driver2.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[4]/div[2]')
print("연령등급 : " + age.text)

publisher=driver2.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[3]/div[2]')
print("출판사 : " + publisher.text)

time.sleep(1)
driver2.quit()

작품이름 -> 
닥터최태수
작품이름 : 닥터 최태수
독점여부 : 미독점
감상인원 : 190.4만명
제목 : 닥터 최태수
연재일 : 연재 중
작가 : 조석호
장르 : 기다무소설현판
연령등급 : 전체이용가
출판사 : 시나브로
